# 単純パーセプトロン(Simple Perceptron)

## 単純パーセプトロンとは<a name="description"></a>

- 線形回帰の後ろに活性化関数を加えたもの
- 活性化関数は何でもいい

### scikit-learnの実装<a name="implementation"></a>

- scikit-learnのlinear_model.Perceptronは、最適化に確率的勾配降下法を用いるロジスティック回帰(linear_model.LogisticRegressionは座標降下法や準ニュートン法などを使用)なので、厳密にはscikit-learnに単純パーセプトロンの活性化関数を自由にカスタマイズできる実装はない

## 使用方法<a name="example"></a>

- scikit-learn, Keras, Chainer, TensorFlowそれぞれの実装を例示
- 活性化関数にはシグモイド関数(ロジスティック関数)を使用(2クラスロジスティック回帰)
- シグモイド関数とは、$0<p<1$ となる確率 $p$ を返す関数

### データ準備<a name="data"></a>

In [ ]:
import numpy as np
from sklearn.datasets.samples_generator import make_blobs

X, y = make_blobs(n_samples=50, centers=2, random_state=0, cluster_std=0.60)
X, y = X.astype(np.float32), y.astype(np.int32)

### 学習<a name="training"></a>

#### scikit-learn<a name="scikit-learn"></a>

In [ ]:
from sklearn.linear_model import Perceptron

sk_clf = Perceptron(n_iter=200, random_state=0)
sk_clf.fit(X, y)

#### Keras<a name="keras"></a>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

keras_clf = Sequential()
keras_clf.add(Dense(output_dim=1, input_dim=2))
keras_clf.add(Activation('sigmoid'))
keras_clf.compile(optimizer=SGD(lr=0.01), loss='binary_crossentropy')
keras_clf.fit(X, y, batch_size=10, nb_epoch=200, verbose=0)

#### Chainer<a name="chainer"></a>

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.datasets.tuple_dataset import TupleDataset
from chainer.iterators import SerialIterator
from chainer.training import StandardUpdater, Trainer

class SimplePerceptron(chainer.Chain):

    def __init__(self):
        super(SimplePerceptron, self).__init__(
            l1 = L.Linear(2, 1)
        )

    def __call__(self, x):
        h = self.l1(x)

        return h

train_data = TupleDataset(X, y.reshape((-1, 1)))
ch_clf = L.Classifier(SimplePerceptron(), lossfun=F.sigmoid_cross_entropy)
ch_clf.compute_accuracy = False
optimizer = chainer.optimizers.SGD()
optimizer.setup(ch_clf)
train_iter = SerialIterator(train_data, 10)
updater = StandardUpdater(train_iter, optimizer, device=-1)
trainer = Trainer(updater, (1000, 'iteration'))

trainer.run()

#### TensorFlow (learn + slim)<a name="tensorflow"></a>

In [ ]:
import tensorflow as tf
learn = tf.contrib.learn
slim = tf.contrib.slim

def model(x, y):
    y = tf.reshape(y, [-1, 1])
    logits = slim.fully_connected(x, 1, activation_fn=None)
    loss = slim.losses.sigmoid_cross_entropy(logits, y)
    train_op = slim.optimize_loss(loss, slim.get_global_step(), learning_rate=0.01, optimizer='SGD')
    prob = tf.nn.sigmoid(logits)

    return {'class': tf.round(prob), 'prob': prob}, loss, train_op

tf_clf = learn.SKCompat(learn.Estimator(model_fn=model))
tf_clf.fit(x=X, y=y, steps=1000, batch_size=10)

### 結果の可視化<a name="visualization"></a>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

margin = .5
feature1, feature2 = X[:, 0], X[:, 1]
x_min, x_max = feature1.min() - margin, feature1.max() + margin
y_min, y_max = feature2.min() - margin, feature2.max() + margin
resolution = min(x_max - x_min, y_max - y_min) / 400

xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution), np.arange(y_min, y_max, resolution))
grid = np.c_[xx.ravel(), yy.ravel()].astype(np.float32)

# scikit-learn
probas = sk_clf.decision_function(grid)
# Keras
# probas = keras_clf.predict_proba(grid, verbose=0)
# Chainer
# probas = F.sigmoid(ch_clf.predictor(grid)).data
# TensorFlow
# probas = tf_clf.predict(grid)['prob']

plt.figure(figsize=(4, 4))

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())

plt.pcolormesh(xx, yy, probas.reshape(xx.shape), cmap='RdBu', alpha=0.6)
plt.scatter(feature1, feature2, c=y, cmap=ListedColormap(['#FF0000', '#0000FF']))

plt.show()